In [12]:
import pandas as pd

# Reload the datasets with correct header row
df_3m = pd.read_excel("US3M.xlsx", sheet_name=1)
df_1y = pd.read_excel("US1Y.xlsx", sheet_name=1)
df_5y = pd.read_excel("US5Y.xlsx", sheet_name=1)
df_10y = pd.read_excel("US10Y.xlsx", sheet_name=1)


# Rename columns uniformly
df_3m.columns = ['date', '3m']
df_1y.columns = ['date', '1y']
df_5y.columns = ['date', '5y']
df_10y.columns = ['date', '10y']

# Convert 'date' column to datetime format
for df in [df_3m, df_1y, df_5y, df_10y]:
    df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Drop rows with missing dates or yields
for df in [df_3m, df_1y, df_5y, df_10y]:
    df.dropna(inplace=True)

# Merge dataframes on date
df_merged = df_10y.merge(df_1y, on='date') \
                  .merge(df_3m, on='date') \
                  .merge(df_5y, on='date')

# Sort by date
df_merged = df_merged.sort_values("date").reset_index(drop=True)

# Set 'date' as index to match your China export format
yield_ts = df_merged.set_index('date')

# Export to CSV
yield_ts.to_csv('us_yield_timeseries_3m_1y_5y_10y.csv')
print(" Combined yield time series saved as 'us_yield_timeseries_3m_1y_5y_10y.csv'.")

# Print time coverage
print(" Year coverage:", yield_ts.index.min().date(), "→", yield_ts.index.max().date())


 Combined yield time series saved as 'us_yield_timeseries_3m_1y_5y_10y.csv'.
 Year coverage: 2018-01-02 → 2024-12-31


In [13]:

import pandas as pd

file_path = "us_yield_timeseries_3m_1y_5y_10y.csv"
df = pd.read_csv(file_path, parse_dates=["date"], index_col="date")

# Compute simple yield differences (Δyield)
daily_yield_changes = df.diff().dropna()

daily_yield_changes.to_csv("us_daily_yield_changes_3m_1y_5y_10y.csv")


In [16]:
import pandas as pd
from arch import arch_model
import matplotlib.pyplot as plt

# Load your ΔYield data
df = pd.read_csv("us_daily_yield_changes_3m_1y_5y_10y.csv", index_col="date", parse_dates=True)

# Loop through each maturity and fit GARCH(1,1)
garch_results = {}

for col in df.columns:
    print(f"\n Fitting GARCH(1,1) for: {col}")
    
    # Drop missing values
    series = df[col].dropna()
    
    # Fit GARCH(1,1) with Student's t-distribution
    model = arch_model(series, vol='GARCH', p=1, q=1, dist='t')
    result = model.fit(disp='off')
    
    # Store result
    garch_results[col] = result

    # Print summary
    print(result.summary())



 Fitting GARCH(1,1) for: 10y
                        Constant Mean - GARCH Model Results                         
Dep. Variable:                          10y   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                            GARCH   Log-Likelihood:                2694.47
Distribution:      Standardized Student's t   AIC:                          -5378.95
Method:                  Maximum Likelihood   BIC:                          -5351.61
                                              No. Observations:                 1749
Date:                      Thu, Apr 17 2025   Df Residuals:                     1748
Time:                              17:58:50   Df Model:                            1
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-------------------------------------------------

/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.003277. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002148. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


                        Constant Mean - GARCH Model Results                         
Dep. Variable:                           3m   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                            GARCH   Log-Likelihood:                2627.89
Distribution:      Standardized Student's t   AIC:                          -5245.78
Method:                  Maximum Likelihood   BIC:                          -5218.44
                                              No. Observations:                 1749
Date:                      Thu, Apr 17 2025   Df Residuals:                     1748
Time:                              17:58:50   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu  

/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001176. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.003634. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [17]:
import pandas as pd
from arch import arch_model

# Load your ΔYield data
df = pd.read_csv("us_daily_yield_changes_3m_1y_5y_10y.csv", index_col="date", parse_dates=True)

maturities = ['3m', '1y', '5y', '10y']
distributions = ['normal', 't', 'ged']

# Store all results
records = []

for maturity in maturities:
    series = df[maturity].dropna()
    
    for dist in distributions:
        model = arch_model(series, vol='GARCH', p=1, q=1, dist=dist)
        res = model.fit(disp='off')

        alpha = res.params.get('alpha[1]', None)
        beta = res.params.get('beta[1]', None)
        shape_param = res.params.get('nu', res.params.get('lambda', None))  # nu for t, lambda for GED

        records.append({
            'Maturity': maturity,
            'Distribution': dist.upper(),
            'LogLik': res.loglikelihood,
            'AIC': res.aic,
            'BIC': res.bic,
            'Alpha (α)': alpha,
            'Beta (β)': beta,
            'Tail Param (ν or κ)': shape_param
        })

# Create DataFrame for comparison
results_table = pd.DataFrame.from_records(records)
results_table = results_table.sort_values(['Maturity', 'Distribution'])

# Display the final result
print(" GARCH(1,1) Fit Comparison for All Maturities:\n")
display(results_table)


/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001176. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001176. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a

 GARCH(1,1) Fit Comparison for All Maturities:



/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.003634. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.003634. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a

,Maturity,Distribution,LogLik,AIC,BIC,Alpha (α),Beta (β),Tail Param (ν or κ)
11,10y,GED,2689.367992,-5368.735984,-5341.401987,0.078529,0.910637,1.656144
9,10y,NORMAL,2680.940339,-5353.880678,-5332.013480,0.078612,0.910293,NaN
10,10y,T,2694.474051,-5378.948102,-5351.614104,0.079235,0.910050,11.327996
5,1y,GED,3234.407183,-6458.814367,-6431.480370,0.209100,0.771170,3.187971
3,1y,NORMAL,3652.085518,-7296.171036,-7274.303838,0.200000,0.780000,NaN
4,1y,T,2341.443919,-4672.887837,-4645.553840,0.000012,0.000245,21.291171
2,3m,GED,4205.646696,-8401.293393,-8373.959395,0.200533,0.779448,1.759373
0,3m,NORMAL,4160.079380,-8312.158759,-8290.291561,0.200000,0.780000,NaN
1,3m,T,2627.888438,-5245.776876,-5218.442879,0.513693,0.401400,219.919864
8,5y,GED,2719.987211,-5429.974422,-5402.640425,0.100004,0.879996,1.499998


In [18]:
import pandas as pd

# Load the merged US yield data (should have 'date' and '5y')
df = pd.read_csv("us_yield_timeseries_3m_1y_5y_10y.csv", parse_dates=["date"])
df = df.sort_values("date").reset_index(drop=True)

# Compute first difference of 5Y yield
df["5y_ret"] = df["5y"].diff()
df = df.dropna()

# Define regimes
df["regime"] = "Recovery"  # default
df.loc[df["date"] < "2020-03-11", "regime"] = "Pre-COVID"
df.loc[(df["date"] >= "2020-03-11") & (df["date"] <= "2022-12-31"), "regime"] = "COVID"

# Confirm the structure
print(df[["date", "5y", "5y_ret", "regime"]].head())
print(df["regime"].value_counts())


        date    5y  5y_ret     regime
1 2018-01-03  2.25    0.00  Pre-COVID
2 2018-01-04  2.27    0.02  Pre-COVID
3 2018-01-05  2.29    0.02  Pre-COVID
4 2018-01-08  2.29    0.00  Pre-COVID
5 2018-01-09  2.33    0.04  Pre-COVID
regime
COVID        704
Pre-COVID    545
Recovery     500
Name: count, dtype: int64


In [19]:
from arch import arch_model
import pandas as pd

# Define model types
model_types = {
    "EGARCH": {"vol": "EGARCH"},
    "GJR": {"vol": "GARCH", "o": 1}
}

# Create output container
results_table = []

# Loop through each regime and model
for regime in ["Pre-COVID", "COVID", "Recovery"]:
    regime_data = df[df["regime"] == regime]["5y_ret"].dropna()
    
    for model_name, model_args in model_types.items():
        print(f"\n🔍 Fitting {model_name}(1,1) on US 5Y ({regime})")
        
        model = arch_model(regime_data, mean="Zero", p=1, q=1, dist="ged", **model_args)
        result = model.fit(disp="off")
        params = result.params

        # Extract core volatility params
        alpha = params.get("alpha[1]", None)
        beta = params.get("beta[1]", None)
        gamma = params.get("gamma[1]", None)  # EGARCH
        gamma_gjr = params.get("eta[1]", None)  # GJR-GARCH
        tail_param = params.get("nu", None) or params.get("lambda", None)

        results_table.append({
            "Model": model_name,
            "Regime": regime,
            "LogLik": result.loglikelihood,
            "AIC": result.aic,
            "BIC": result.bic,
            "Alpha (α)": alpha,
            "Beta (β)": beta,
            "Gamma (γ)": gamma if gamma is not None else gamma_gjr,
            "Tail Param (κ)": tail_param
        })

        print(result.summary())

# Convert to DataFrame for export or comparison
summary_df = pd.DataFrame(results_table)
print("\n📊 GARCH-family model summary (US 5Y):")
print(summary_df)

# Optional: export to CSV
summary_df.to_csv("us_5y_egarch_gjr_results.csv", index=False)
print("\n✅ Exported results to 'us_5y_egarch_gjr_results.csv'")



🔍 Fitting EGARCH(1,1) on US 5Y (Pre-COVID)
                             Zero Mean - EGARCH Model Results                             
Dep. Variable:                             5y_ret   R-squared:                       0.000
Mean Model:                             Zero Mean   Adj. R-squared:                  0.002
Vol Model:                                 EGARCH   Log-Likelihood:                967.164
Distribution:      Generalized Error Distribution   AIC:                          -1926.33
Method:                        Maximum Likelihood   BIC:                          -1909.12
                                                    No. Observations:                  545
Date:                            Thu, Apr 17 2025   Df Residuals:                      545
Time:                                    18:06:31   Df Model:                            0
                              Volatility Model                             
                 coef    std err          t      P>|t|     95

/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001835. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001835. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a

                             Zero Mean - EGARCH Model Results                             
Dep. Variable:                             5y_ret   R-squared:                       0.000
Mean Model:                             Zero Mean   Adj. R-squared:                  0.001
Vol Model:                                 EGARCH   Log-Likelihood:                1174.70
Distribution:      Generalized Error Distribution   AIC:                          -2341.41
Method:                        Maximum Likelihood   BIC:                          -2323.18
                                                    No. Observations:                  704
Date:                            Thu, Apr 17 2025   Df Residuals:                      704
Time:                                    18:06:31   Df Model:                            0
                             Volatility Model                             
                 coef    std err          t      P>|t|    95.0% Conf. Int.
-------------------------------

/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.00351. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a1b970b-0a9f-430f-ab6d-5c5f2e3979db/.local/lib/python3.10/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.005734. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/home/3a1


✅ Exported results to 'us_5y_egarch_gjr_results.csv'
